In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import random
import warnings
warnings.filterwarnings('ignore')

# 모델 생성
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report


# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
df = pd.read_excel("Churn_final.xlsx")
df.head()

,Customer ID,Age,Number of Dependents,Membership,Satisfaction Score,Tech services,Streaming services,Combined Product,Contract,Tenure in Months,Monthly Charge,Total Revenue,Churn Value
0,8779-QRDMV,78,0,None,2,1,1,2,Month-to-Month,1,39.65,59.65,1
1,7495-OOKFY,74,1,Offer E,5,1,0,1,Month-to-Month,8,80.65,1024.10,1
2,1658-BYGOY,71,3,Offer D,3,0,2,1,Month-to-Month,18,95.45,1910.88,1
3,4598-XLKNJ,78,1,Offer C,3,2,2,1,Month-to-Month,25,98.50,2995.07,1
4,4846-WHAFZ,80,1,Offer C,1,0,0,1,Month-to-Month,37,76.50,3102.36,1


In [3]:
train_df = df.copy()
train_df = train_df.iloc[:,1:] # Customer ID 제외

# 범주형 컬럼 One-Hot Encoding
encoding_df = pd.get_dummies(train_df, columns=['Membership', 'Contract'])

# 학습/테스트셋 분리
y_target = encoding_df['Churn Value']
X_data = encoding_df.drop(['Churn Value'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=156)

# 학습 데이터셋(수치형 컬럼) 정규화
## Initialize variable
X_train_origin, X_test_origin, y_train_origin, y_test_origin = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()

## Numeric Only
Numeric_column_list = []
for i in range(len(X_data.columns)):
    if X_data[X_data.columns[i]].dtype == 'float64' or X_data[X_data.columns[i]].dtype == 'int64':
        Numeric_column_list.append(X_data.columns[i])

numeric_train_data, numeric_test_data = X_train[Numeric_column_list], X_test[Numeric_column_list]

scaler = MinMaxScaler()

X_train[Numeric_column_list] = scaler.fit_transform(numeric_train_data)
X_test[Numeric_column_list] = scaler.transform(numeric_test_data)


# 선정한 모델로 학습
best_model = SGDClassifier(random_state=42, alpha=0.001, loss='modified_huber',
                           max_iter=100, penalty='l1', tol=1e-05)

best_model.fit(X_train, y_train)

SGDClassifier(alpha=0.001, loss='modified_huber', max_iter=100, penalty='l1',
              random_state=42, tol=1e-05)

In [4]:
import joblib
joblib.dump(best_model, './SGD_model.pkl')

['./SGD_model.pkl']

In [5]:
# 저장한 모델 불러와 변수에 담기
loaded_model = joblib.load('./SGD_model.pkl')
loaded_model

SGDClassifier(alpha=0.001, loss='modified_huber', max_iter=100, penalty='l1',
              random_state=42, tol=1e-05)

In [6]:
# 불러온 모델로 정확도 계산
score = loaded_model.score(X_train, y_train)
score

0.8336883209087682

In [7]:
def churn_prediction(df):
    # 예측 결과 저장

    result_df = df.copy()
    result_df = result_df.iloc[:, 1:-1] # Customer ID, Churn Value 제외

    ## 범주형 컬럼 One-Hot Encoding
    encoding_test_data = pd.get_dummies(result_df, columns=['Membership', 'Contract'])

    ## 정규화 (Numeric Only)
    test_Numeric_column_list = []
    for i in range(len(encoding_test_data.columns)):
        if encoding_test_data[encoding_test_data.columns[i]].dtype == 'float64' or encoding_test_data[encoding_test_data.columns[i]].dtype == 'int64':
            test_Numeric_column_list.append(encoding_test_data.columns[i])

    test_numeric_data = encoding_test_data[test_Numeric_column_list]

    scaler = MinMaxScaler()

    encoding_test_data[test_Numeric_column_list] = scaler.fit_transform(test_numeric_data)
    final_test_data = encoding_test_data


    # 최적의 모델을 사용하여 테스트 데이터의 클래스 확률을 예측합니다.
    predicted_probabilities = loaded_model.predict_proba(final_test_data)

    # 결과를 데이터프레임에 추가
    df['이탈확률'] = predicted_probabilities[:, 1]*100  # 클래스 1의 확률을 선택
    
    return df

In [8]:
df = pd.read_excel("Churn_final.xlsx")
df.head()

,Customer ID,Age,Number of Dependents,Membership,Satisfaction Score,Tech services,Streaming services,Combined Product,Contract,Tenure in Months,Monthly Charge,Total Revenue,Churn Value
0,8779-QRDMV,78,0,None,2,1,1,2,Month-to-Month,1,39.65,59.65,1
1,7495-OOKFY,74,1,Offer E,5,1,0,1,Month-to-Month,8,80.65,1024.10,1
2,1658-BYGOY,71,3,Offer D,3,0,2,1,Month-to-Month,18,95.45,1910.88,1
3,4598-XLKNJ,78,1,Offer C,3,2,2,1,Month-to-Month,25,98.50,2995.07,1
4,4846-WHAFZ,80,1,Offer C,1,0,0,1,Month-to-Month,37,76.50,3102.36,1


In [9]:
result = churn_prediction(df)
result

,Customer ID,Age,Number of Dependents,Membership,Satisfaction Score,Tech services,Streaming services,Combined Product,Contract,Tenure in Months,Monthly Charge,Total Revenue,Churn Value,이탈확률
0,8779-QRDMV,78,0,None,2,1,1,2,Month-to-Month,1,39.65,59.65,1,65.700984
1,7495-OOKFY,74,1,Offer E,5,1,0,1,Month-to-Month,8,80.65,1024.10,1,52.182243
2,1658-BYGOY,71,3,Offer D,3,0,2,1,Month-to-Month,18,95.45,1910.88,1,40.846450
3,4598-XLKNJ,78,1,Offer C,3,2,2,1,Month-to-Month,25,98.50,2995.07,1,56.178603
4,4846-WHAFZ,80,1,Offer C,1,0,0,1,Month-to-Month,37,76.50,3102.36,1,62.192832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,30,0,None,5,0,0,4,Two Year,72,21.15,3039.53,0,0.000000
7039,6840-RESVB,38,2,Offer C,5,3,2,2,One Year,24,84.80,2807.47,0,0.000000
7040,2234-XADUH,30,2,None,3,2,2,1,One Year,72,103.20,9453.04,0,0.000000
7041,4801-JZAZL,32,2,None,2,1,0,2,Month-to-Month,11,29.60,319.21,0,26.726435


In [69]:
import numpy as np
import pandas as pd

# 사용자로부터 입력 받기
input_labels_1 = ["Age", "Number of Dependents", "Satisfaction Score", "Tech services",
                "Streaming services", "Combined Product", "Tenure in Months", "Monthly Charge", "Total Revenue"]
input_labels_2 = ['Membership_None', 'Membership_Offer A', 'Membership_Offer B', 'Membership_Offer C', 'Membership_Offer D', 'Membership_Offer E', 'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year']

input_values_1 = []
for label in input_labels_1:
    value = float(input(f"{label}를 입력하세요: "))
    input_values_1.append(value)

# '멤버십 단계' 및 '계약 형태'
Membership = input("멤버십 단계 (None, Offer A, Offer B, Offer C, Offer D, Offer E 중 하나)를 입력하세요: ")
Contract = input("계약 형태 (Month-to-Month, One Year, Two Year 중 하나)를 입력하세요: ")

# 더미 변수 생성
membership_dummy = [0, 0, 0, 0, 0, 0]  # None, Offer A, Offer B, Offer C, Offer D, Offer E에 대한 더미 변수
contract_dummy = [0, 0, 0]  # Month-to-Month, One Year, Two Year에 대한 더미 변수

if Membership == "None":
    membership_dummy[0] = 1
elif Membership == "Offer A":
    membership_dummy[1] = 1
elif Membership == "Offer B":
    membership_dummy[2] = 1
elif Membership == "Offer C":
    membership_dummy[3] = 1
elif Membership == "Offer D":
    membership_dummy[4] = 1
elif Membership == "Offer E":
    membership_dummy[5] = 1
    
# 같은 방식으로 Contract에 대한 더미 변수 설정
if Contract == "Month-to-Month":
    contract_dummy[0] = 1
elif Contract == "One Year":
    contract_dummy[1] = 1
elif Contract == "Two Year":
    contract_dummy[2] = 1
    
input_values_2 = input_values_1 + membership_dummy + contract_dummy
input_values_2

from sklearn.preprocessing import MinMaxScaler

# 입력 데이터를 선택
input_labels = input_labels_1 + input_labels_2
input_data = pd.DataFrame([input_values_2], columns=input_labels)

# Min-Max 스케일러 생성 - 스케일러 통일 필요
scaler = MinMaxScaler()

# 데이터 스케일링
scaled_data = scaler.fit_transform(input_data)
scaled_data


# 입력 데이터를 Numpy 배열로 변환
customer_data = np.array(scaled_data, dtype=np.float32)

# 예측 수행
y_customer_pred = best_model.predict(customer_data)

# 예측 확률 계산
y_customer_prob = best_model.predict_proba(customer_data)

# 예측 결과 출력
if y_customer_prob[0][1] >= 0.6:
    print("\n고위험 이탈그룹")
elif y_customer_prob[0][1] >= 0.4:
    print("\n잠재적 이탈그룹")
else:
    print("\n안정그룹")

print("이탈확률:", y_customer_prob[0][1])

# 78 0 3 1 1 2 1 39.65 59.65 None Month-to-Month 해지

# 스케일링된 데이터를 DataFrame으로 변환 (선택 사항)
#columns = ["Age", "Number of Dependents", "Satisfaction Score", "Tech services", "Streaming services", "Combined Product", "Tenure in Months", "Monthly Charge", "Total Revenue", 
#          'Membership_None', 'Membership_Offer A', 'Membership_Offer B', 'Membership_Offer C', 'Membership_Offer D', 'Membership_Offer E', 'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year']


Age를 입력하세요: 1
Number of Dependents를 입력하세요: 1
Satisfaction Score를 입력하세요: 1
Tech services를 입력하세요: 1
Streaming services를 입력하세요: 1
Combined Product를 입력하세요: 1
Tenure in Months를 입력하세요: 1
Monthly Charge를 입력하세요: 1
Total Revenue를 입력하세요: 1
멤버십 단계 (None, Offer A, Offer B, Offer C, Offer D, Offer E 중 하나)를 입력하세요: None
계약 형태 (Month-to-Month, One Year, Two Year 중 하나)를 입력하세요: Month-to-Month

안정그룹
이탈확률: 0.33552149552604316


In [70]:
import numpy as np
import pandas as pd

# 원본 입력 값
original_input_values = input_values_2.copy()

# 피처 라벨
features_labels = ["Age", "Number of Dependents", "Satisfaction Score", "Tech services", "Streaming services", "Combined Product", "Tenure in Months", "Monthly Charge", "Total Revenue", 
          'Membership_None', 'Membership_Offer A', 'Membership_Offer B', 'Membership_Offer C', 'Membership_Offer D', 'Membership_Offer E', 'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year']

# 변경할 피처 인덱스
feature_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

# 그룹 변경을 위한 임계값
threshold = 0.5

# 피처 인덱스를 반복하고 변경사항을 적용
for feature_index in feature_indices:
    # 원본 입력 값을 복사
    modified_input_values = original_input_values.copy()

    # 피처 인덱스를 기반으로 변경 사항 적용 (먼저 나온것만 반영됨. 전부 나오게?)
    if 1 <= feature_index <= 6:
        # 부양 가족 수, 만족도 점수, 기술 서비스, 스트리밍 서비스, 결합 상품, 계약 기간(개월): +/- 1 변경
        modified_input_values[feature_index] += 1
        if modified_input_values[feature_index] < 0:
            modified_input_values[feature_index] = 0
        #modified_input_values[feature_index + 8] = 1  # 해당하는 멤버십 더미 변수를 1로 설정?
    elif 7 <= feature_index <= 8:
        # 월 요금, 총 수익: +/- 0.1 변경
        modified_input_values[feature_index] -= 0.1
        if modified_input_values[feature_index] < 0:
            modified_input_values[feature_index] = 0
    elif 9 <= feature_index <= 14:
        # 멤버십 (원-핫 인코딩): 다음 멤버십 레벨로 변경
        if modified_input_values[feature_index] == 14:
            modified_input_values[feature_index] = 0  # 현재 멤버십을 0으로 설정
            modified_input_values[feature_index + 1] = 1  # 다음 멤버십을 1로 설정
    elif 15 <= feature_index <= 17:
        # 계약 (원-핫 인코딩): 다음 계약 유형으로 변경
        if modified_input_values[feature_index] == 17:
            modified_input_values[feature_index] = 0  # 현재 계약을 0으로 설정
            modified_input_values[feature_index +1] = 1  # 다음 계약을 1로 설정

    # 변경된 입력 값을 위한 DataFrame 생성
    modified_input_data = pd.DataFrame([modified_input_values], columns=features_labels)

    # 변경된 데이터를 스케일링
    modified_scaled_data = scaler.transform(modified_input_data)
    modified_customer_data = np.array(modified_scaled_data, dtype=np.float32)

    # 변경된 데이터로 예측 수행
    modified_customer_prob_changed = best_model.predict_proba(modified_customer_data)

    if modified_customer_prob_changed[0][1] < threshold:
        print(f"{features_labels[feature_index]} 값을 {modified_input_values[feature_index]}로 변경하면 '안정그룹'이 될 수 있습니다.")
        
# 예측 결과 출력
if modified_customer_prob_changed[0][1] >= 0.6:
    print("")
    print("고위험 이탈그룹")
elif modified_customer_prob_changed[0][1] >= 0.4:
    print("")
    print("잠재적 이탈그룹")
else:
    print("")
    print("안정그룹")

print("")
print("이탈확률:", modified_customer_prob_changed[0][1])


Number of Dependents 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Satisfaction Score 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Tech services 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Streaming services 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Combined Product 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Tenure in Months 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Monthly Charge 값을 0.9로 변경하면 '안정그룹'이 될 수 있습니다.
Total Revenue 값을 0.9로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_None 값을 1로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer A 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer B 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer C 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer D 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer E 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Contract_Month-to-Month 값을 1로 변경하면 '안정그룹'이 될 수 있습니다.
Contract_One Year 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Contract_Two Year 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.

안정그룹

이탈확률: 0.33552149552604316


In [71]:
#이탈 확률이 0.3으로 똑같음?, 사용파일 Churn_final.xlsx로 바뀜.

In [72]:
import numpy as np
import pandas as pd

# 원본 입력 값
original_input_values = input_values_2.copy()

# 피처 라벨
features_labels = ["Age", "Number of Dependents", "Satisfaction Score", "Tech services", "Streaming services", "Combined Product", "Tenure in Months", "Monthly Charge", "Total Revenue", 
          'Membership_None', 'Membership_Offer A', 'Membership_Offer B', 'Membership_Offer C', 'Membership_Offer D', 'Membership_Offer E', 'Contract_Month-to-Month', 'Contract_One Year', 'Contract_Two Year']

# 변경할 피처 인덱스
feature_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

# 그룹 변경을 위한 임계값
threshold = 0.5

# 모든 변경된 입력 값에 대한 결과 저장
modified_results = []

# 피처 인덱스를 반복하고 변경사항을 적용
for feature_index in feature_indices:
    # 원본 입력 값을 복사
    modified_input_values = original_input_values.copy()

    # 피처 인덱스를 기반으로 변경 사항 적용 (먼저 나온것만 반영됨. 전부 나오게?)
    if 1 <= feature_index <= 6:
        # 부양 가족 수, 만족도 점수, 기술 서비스, 스트리밍 서비스, 결합 상품, 계약 기간(개월): +/- 1 변경
        modified_input_values[feature_index] += 1
        if modified_input_values[feature_index] < 0:
            modified_input_values[feature_index] = 0
        #modified_input_values[feature_index + 8] = 1  # 해당하는 멤버십 더미 변수를 1로 설정?
    elif 7 <= feature_index <= 8:
        # 월 요금, 총 수익: +/- 0.1 변경
        modified_input_values[feature_index] -= 0.1
        if modified_input_values[feature_index] < 0:
            modified_input_values[feature_index] = 0
    elif 9 <= feature_index <= 14:
        # 멤버십 (원-핫 인코딩): 다음 멤버십 레벨로 변경
        if modified_input_values[feature_index] == 14:
            modified_input_values[feature_index] = 0  # 현재 멤버십을 0으로 설정
            modified_input_values[feature_index + 1] = 1  # 다음 멤버십을 1로 설정
    elif 15 <= feature_index <= 17:
        # 계약 (원-핫 인코딩): 다음 계약 유형으로 변경
        if modified_input_values[feature_index] == 17:
            modified_input_values[feature_index] = 0  # 현재 계약을 0으로 설정
            modified_input_values[feature_index +1] = 1  # 다음 계약을 1로 설정

    # 변경된 입력 값을 위한 DataFrame 생성
    modified_input_data = pd.DataFrame([modified_input_values], columns=features_labels)

    # 변경된 데이터를 스케일링
    modified_scaled_data = scaler.transform(modified_input_data)
    modified_customer_data = np.array(modified_scaled_data, dtype=np.float32)

    # 변경된 데이터로 예측 수행
    modified_customer_prob_changed = best_model.predict_proba(modified_customer_data)

    modified_results.append((feature_index, modified_input_values, modified_customer_prob_changed))

# 모든 변경된 결과 출력
for result in modified_results:
    feature_index, modified_input_values, modified_customer_prob_changed = result
    if modified_customer_prob_changed[0][1] < threshold:
        print(f"{features_labels[feature_index]} 값을 {modified_input_values[feature_index]}로 변경하면 '안정그룹'이 될 수 있습니다.")

# 마지막 결과 출력
if modified_results[-1][-1][0][1] >= 0.6:
    print("\n고위험 이탈그룹")
elif modified_results[-1][-1][0][1] >= 0.4:
    print("\n잠재적 이탈그룹")
else:
    print("\n안정그룹")

print("\n이탈확률:", modified_results[-1][-1][0][1])

Number of Dependents 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Satisfaction Score 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Tech services 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Streaming services 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Combined Product 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Tenure in Months 값을 2.0로 변경하면 '안정그룹'이 될 수 있습니다.
Monthly Charge 값을 0.9로 변경하면 '안정그룹'이 될 수 있습니다.
Total Revenue 값을 0.9로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_None 값을 1로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer A 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer B 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer C 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer D 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Membership_Offer E 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Contract_Month-to-Month 값을 1로 변경하면 '안정그룹'이 될 수 있습니다.
Contract_One Year 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.
Contract_Two Year 값을 0로 변경하면 '안정그룹'이 될 수 있습니다.

안정그룹

이탈확률: 0.33552149552604316
